In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output

# Data processing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# CRUD import
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
shelter = AnimalShelter(
    user="aacuser",
    password="SNHU1234",
    host="nv-desktop-services.apporto.com",
    port=33544,
    db_name="AAC",
    collection_name="animals"
)

# Get all records and drop _id column
df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard – Joseph Dumke'))),
    html.Hr(),

    # DataTable
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                 for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action='native',
        sort_action='native',
        page_action='native',
        sort_mode='multi',
        column_selectable='single',
        row_selectable='single',
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],  # ← Select first row by default
        page_size=10,
        page_current=0,
    ),

    html.Br(),
    html.Hr(),

    # Map container
    html.Div(id='map-id', className='col s12 m6')
])

#############################################
# Interaction Between Components / Controller
#############################################

# Highlight selected column (guard against None)
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', 'selected_columns')
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': {'column_id': col},
        'background_color': '#D2F3FF'
    } for col in selected_columns]

# Fixed update_map function
def update_map(viewData, index):
    # Build DataFrame from the table’s current view (list of dicts)
    if viewData:
        dff = pd.DataFrame(viewData)
    else:
        dff = df.copy()

    # Determine row index (default to 0)
    if not index:
        row = 0
    else:
        row = index[0]

    # Extract latitude/longitude using correct column names
    lat = dff.loc[row, 'location_lat']
    lon = dff.loc[row, 'location_long']

    # Tooltip shows breed; popup shows name
    breed = dff.loc[row, 'breed']
    name  = dff.loc[row, 'name']

    # Return the map (as a single-item list) with marker
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[lat, lon],
            zoom=12,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(name)
                        ])
                    ]
                )
            ]
        )
    ]

# Callback to render the map
@app.callback(
    Output('map-id', 'children'),
    [
        Input('datatable-id', 'derived_virtual_data'),
        Input('datatable-id', 'derived_virtual_selected_rows')
    ]
)
def display_map(viewData, selected_rows):
    return update_map(viewData, selected_rows)

# Run the app inside Jupyter
app.run_server(mode='inline', port=33544, debug=True)
